In [39]:
#generate i2c bus traffic for developing i2c slave

SDA_stream=[]
SCL_stream=[]

#timing in ns for 100khz i2c
TimeBit=250 # time for one bit in ns
TimeRise=1000
TimeFall=300
TimeHigh=4000
TimeLow=4700
TimeStart=4000 # start and stop bit timing

#https://stackoverflow.com/questions/32284940/python-bit-list-to-byte-list
DeviceAddress=int('0110000',2)

# start bit
def S_bit():
    return int('100',2),int('111',2)

# stop bit
def P_bit():
    return int('011',2),int('111',2)

# scale number as binary string
def scalestr(n):
    string=""
    width=(TimeHigh//TimeBit)+(TimeLow//TimeBit)
    

# write to one register. Accepts only numbers
def writeone():
    string=""
    width=(TimeHigh//TimeBit)+(TimeLow//TimeBit)
    print(width)
    offset=(TimeLow//TimeBit)//2
    string+=str('0')*offset
    string+="_"
    string+=(format(DeviceAddress<<1, '#010b')[2:])
    return string
    
#generate SCL, scale is how many of each "bit"
#def genSCL(scale,length):
#    b=1
#    string=""
#    for i in range(0,length):
#        string+=str(b)*scale
#        if b==1:
#            b=0
#        else:
#            b=1
#    return string

#generate SCL by time settings
def genSCLtime(length):
    b=1
    string=""
    for i in range(0,length):
        string+=str('1')*((TimeRise+TimeHigh)//TimeBit)+str('0')*((TimeFall+TimeLow)//TimeBit)
    return string
    
    

#print(format(5, '#010b')[2:],end="")
#print(format(5, '#010b')[2:])

print(writeone())
print(genSCLtime(4))

34
000000000_01100000
1111111111111111111100000000000000000000111111111111111111110000000000000000000011111111111111111111000000000000000000001111111111111111111100000000000000000000
